<div dir="auto">

  
https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset

In [1]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copy, move
from glob import glob as g
from tqdm.notebook import tqdm

In [2]:
from IPython.display import clear_output
!pip install --upgrade kaggle
clear_output()
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 74 bytes


In [3]:
!kaggle datasets download -d pybear/cats-vs-dogs

 99% 782M/788M [00:06<00:00, 153MB/s]
100% 788M/788M [00:06<00:00, 136MB/s]


In [5]:
!unzip -xq '/content/cats-vs-dogs.zip' #Linux Command


In [6]:
#!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/content/cats-and-dogs.zip"

#!unzip -xq '/content/cats-and-dogs.zip' #Linux Command

count = 0
for name in tqdm(g('/content/PetImages/*/*')):
  
  if not os.path.getsize(name):
    count += 1
    os.remove(name)
print(count, ' empty files removed')

--2023-02-26 14:10:57--  http://kaggle%20competitions%20download%20-c%20dogs-vs-cats/
Resolving kaggle competitions download -c dogs-vs-cats (kaggle competitions download -c dogs-vs-cats)... failed: Name or service not known.
wget: unable to resolve host address ‘kaggle competitions download -c dogs-vs-cats’
[/content/cats-and-dogs.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/cats-and-dogs.zip or
        /content/cats-and-dogs.zip.zip, and cannot find /content/cats-and-dogs.zip.ZIP, period.


0it [00:00, ?it/s]

0  empty files removed


In [6]:
names = os.listdir('/content/PetImages/Cat')
print(len(names))
print(names[:5])

12501
['2721.jpg', '8333.jpg', '10536.jpg', '2816.jpg', '10896.jpg']


In [7]:
names = os.listdir('/content/PetImages/Dog')
print(len(names))
print(names[:5])

12501
['2721.jpg', '8333.jpg', '10536.jpg', '2816.jpg', '10896.jpg']


In [8]:
g('/content/PetImages/*/*')[:5]

['/content/PetImages/Dog/2721.jpg',
 '/content/PetImages/Dog/8333.jpg',
 '/content/PetImages/Dog/10536.jpg',
 '/content/PetImages/Dog/2816.jpg',
 '/content/PetImages/Dog/10896.jpg']

In [9]:
train_cats_dir = '/content/cats-v-dogs/training/cats'
train_dogs_dir = '/content/cats-v-dogs/training/dogs'
test_cats_dir = '/content/cats-v-dogs/testing/cats'
test_dogs_dir = '/content/cats-v-dogs/testing/dogs'
to_create = [
            train_cats_dir,
            train_dogs_dir,
            test_cats_dir,
            test_dogs_dir
]

for dir in to_create:
  os.makedirs(dir)

In [10]:
all_cats = sorted(g("/content/PetImages/Cat/*"))
all_dogs = sorted(g("/content/PetImages/Dog/*"))

In [11]:
print(len(all_cats))
print(len(all_dogs))

12501
12501


In [12]:
from sklearn.model_selection import train_test_split
dogs_train, dogs_test = train_test_split(all_dogs, test_size = 0.1, random_state = 101, shuffle = True)

In [13]:
len(dogs_train) + len(dogs_test)

12501

In [14]:
cats_train, cats_test = train_test_split(all_cats, test_size = 0.1, random_state = 101, shuffle = True)

In [15]:
for name in tqdm(dogs_train):
  copy(name , os.path.join(train_dogs_dir , name.split('/')[-1]))
for name in tqdm(cats_train):
  copy(name , os.path.join(train_cats_dir , name.split('/')[-1]))
  
for name in tqdm(dogs_test):
  copy(name , os.path.join(test_dogs_dir , name.split('/')[-1]))
for name in tqdm(cats_test):
  copy(name , os.path.join(test_cats_dir , name.split('/')[-1]))

  0%|          | 0/11250 [00:00<?, ?it/s]

  0%|          | 0/11250 [00:00<?, ?it/s]

  0%|          | 0/1251 [00:00<?, ?it/s]

  0%|          | 0/1251 [00:00<?, ?it/s]

In [16]:
print(len(os.listdir(train_dogs_dir)))
print(len(os.listdir(train_cats_dir)))
print(len(os.listdir(test_cats_dir)))
print(len(os.listdir(test_dogs_dir)))

# expected output
# 11250
# 11250
# 1250
# 1250

11250
11250
1251
1251


In [17]:
TRAINING_DIR = '/content/cats-v-dogs/training'
train_datagen = ImageDataGenerator(rescale=1 / 255)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size=64,
    class_mode='binary',
    target_size=(150, 150)
)

VALIDATION_DIR = "/content/cats-v-dogs/testing"
validation_datagen = ImageDataGenerator(rescale=1 / 255)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    batch_size=64,
    class_mode='binary',
    target_size=(150, 150)
)

# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

Found 22498 images belonging to 2 classes.
Found 2502 images belonging to 2 classes.


In [18]:
gen_output = next(train_generator)

In [19]:
type(gen_output)

tuple

In [20]:
len(gen_output)

2

In [21]:
gen_output[0].shape

(64, 150, 150, 3)

In [22]:
gen_output[1]

array([1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0.], dtype=float32)

In [23]:
print(validation_generator.class_indices)

{'cats': 0, 'dogs': 1}


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
model = Sequential()
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (150,150,3)))
model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(Dropout(0.3))
model.add(MaxPool2D(2,2))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(Dropout(0.3))
model.add(MaxPool2D(2,2))

model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(Dropout(0.3))
model.add(MaxPool2D(2,2))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))

In [28]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

...

In [29]:
from tensorflow.keras.optimizers.legacy import Adam


In [ ]:
print('Num Params : ',model.count_params())
history = model.fit(train_generator,epochs=30,validation_data=validation_generator)

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

# Desired output. Charts with training and validation metrics. No crash :)

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
    # predicting images
    path = '/content/' + fn
    img = image.load_img(path, target_size=(150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0) / 255.
    classes = model.predict(x, batch_size=10)
    print(classes[0])
    if classes[0]>0.5:
        print(fn + " is a human")
    else:
        print(fn + " is a horse")